In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score
from sklearn.tree import  DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import joblib
from imblearn.combine import SMOTEENN


In [2]:
df = pd.read_csv('telco_churn.csv')

In [3]:
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,1,0,1,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0


### Separating the independent and dependent variable

In [6]:
x = df.drop('Churn', axis=1)
x

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,0,1,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
7028,0,103.20,7362.90,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
7029,0,29.60,346.45,1,0,0,1,0,1,1,...,0,0,1,0,1,0,0,0,0,0
7030,1,74.40,306.60,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [7]:
y = df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

#### TRAIN TEST SPLIT


In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

### Decision Tree classfier

In [9]:
dt_model = DecisionTreeClassifier(criterion= 'gini', random_state=100, max_depth=6, min_samples_leaf=8)

In [10]:
dt_model.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [11]:
y_pred = dt_model.predict(x_test)

In [12]:
y_pred

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [13]:
dt_model.score(x_test, y_test)

0.7746979388770433

In [14]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1023
           1       0.61      0.47      0.53       384

    accuracy                           0.77      1407
   macro avg       0.72      0.68      0.69      1407
weighted avg       0.76      0.77      0.76      1407



#### We can see in above that the accuracy for 1(yes churn) is low because of the majority in not churned customer which makes the data imbalance which can lead to bad accuracy in the model. this is a bad model. so we can handle this either by upsampling or downsampling but if we do downsampling of the majority we could loose important information which is important to train the model. below we will do the down sampling

### To fix this we use smoteenn 

In [15]:
smote = SMOTEENN()
X_resampled1, y_resampled1 = smote.fit_resample(x, y)

In [16]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled1,y_resampled1,test_size=0.2)

In [17]:
def format_classification_report(report):
    lines = report.split('\n')
    header = lines[0]
    table = "\n".join(lines[2:])
    return f"{header}\n{table}"

In [18]:
def churn_analysis_with_best_model(X_train, X_test, y_train, y_test):
    models = {
        "Random Forest": (RandomForestClassifier(n_estimators=100, criterion='gini', random_state=100, max_depth=6, min_samples_leaf=8), {}),
        "Gradient Boosting": (GradientBoostingClassifier(), {'max_depth': [3, 5], 'n_estimators': [100, 200]}),
        "AdaBoost": (AdaBoostClassifier(), {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.5]}),
        "Logistic Regression": (LogisticRegression(max_iter=1000), {'C': [0.1, 1, 10]}),
        "SVM": (SVC(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
        "Naive Bayes": (GaussianNB(), {})
    }
    
    best_f1_score = 0
    best_model = None
    best_model_name = ''
    best_classification_report = ''

    for model_name, (model, param_grid) in models.items():
        if model_name == "Logistic Regression" or model_name == "SVM":
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            clf = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
            clf.fit(X_train_scaled, y_train)
            best_model = clf.best_estimator_
            X_test_used = X_test_scaled
        else:
            clf = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
            clf.fit(X_train, y_train)
            best_model = clf.best_estimator_
            X_test_used = X_test

        y_pred = best_model.predict(X_test_used)
        current_f1_score = f1_score(y_test, y_pred)
        
        if current_f1_score > best_f1_score:
            best_f1_score = current_f1_score
            best_model_name = model_name
            best_classification_report = format_classification_report(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

    # Save the best model using joblib
    if best_model_name:
        best_model_filename = f'best_model_{best_model_name.replace(" ", "_")}.joblib'
        joblib.dump(best_model, best_model_filename)
        print(f"Saved {best_model_name} model as {best_model_filename}")
    
    return best_classification_report

In [19]:
# Usage
best_classification_report = churn_analysis_with_best_model(xr_train, xr_test, yr_train, yr_test)
print(best_classification_report)

Saved Gradient Boosting model as best_model_Gradient_Boosting.joblib
              precision    recall  f1-score   support
    No Churn       0.96      0.94      0.95       515
       Churn       0.95      0.97      0.96       629

    accuracy                           0.96      1144
   macro avg       0.96      0.95      0.95      1144
weighted avg       0.96      0.96      0.96      1144



#### As we observed that after applying smoteenn to handle the imabalance dataset , after handling we got the accuracy improved

In [20]:
## we can further perform the Principal component analysis (PCA)

In [21]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train)
xr_test_pca = pca.transform(xr_test)
explained_variance = pca.explained_variance_ratio_

In [22]:
model=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [23]:
model.fit(xr_train_pca,yr_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [24]:
yr_predict_pca = model.predict(xr_test_pca)

In [25]:
model_score_r_pca = model.score(xr_test_pca, yr_test)

In [26]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test, yr_predict_pca))

0.7195540308747856
              precision    recall  f1-score   support

           0       0.70      0.64      0.67       516
           1       0.73      0.78      0.76       650

    accuracy                           0.72      1166
   macro avg       0.72      0.71      0.71      1166
weighted avg       0.72      0.72      0.72      1166



- With PCA, we couldn't see any better results, hence let's finalise the model which was created by using the function and the best model is gradient boosting, and save the model so that we can use it in a later stage

#### Loading the best Model

In [20]:
model_filename = "best_model_Gradient_Boosting.joblib"  

In [21]:
loaded_model = joblib.load(model_filename)

In [24]:
loaded_model.score(xr_test, yr_test)

0.9108391608391608